# AOC2022

## Day 11 / Part 2 / Monkey in the Middle

Problem Description: https://adventofcode.com/2022/day/11#part2

Input: [Example](aoc2022_day11_example.txt)

### Notes
- This solution uses the unsecure `eval` function which is neat for a coding challenge but should be avoided in a real project!
- Given a naive implementation for computing 10000 rounds, we run into an issue with the continously increasing `worry_level`s. Hence, we compute the product of all test-divisors (`div_prod`) which is used with a modulo operation to keep the `worry_level`s low. 

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 11, part 2."""
import logging
import re
import sys

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day11_example.txt"

In [3]:
ROUNDS = 10000

In [4]:
class Monkey:
    """Monkey in the game."""
    items = []
    op = ""
    test = ["", -1, -1]
    inspected_items = 0

    def pop_item(self):
        """Pop the first item the monkey holds."""
        self.inspected_items += 1
        return self.items.pop(0)

    def eval_op(self, item):
        """Evaluate the monkey's operation."""
        # pylint: disable=W0123,W0612
        old = item
        return eval(self.op[6:])

    def get_next_monkey(self, item):
        """Get the monkey which receives an item."""
        expr, true_mi, false_mi = self.test
        if expr.startswith("divisible"):
            return true_mi if (item % int(expr[13:]) == 0) else false_mi
        raise RuntimeError("unknown test!")

    def __repr__(self):
        """Compute the printable string representation of the monkey."""
        return f"Monkey ({self.items}, {self.op}, {self.test})"

In [5]:
def main():
    """Main function to solve puzzle."""
    monkeys = []
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        lines = [line.rstrip() for line in file_obj.readlines()]
        line_idx = 0
        while line_idx < len(lines):
            line = lines[line_idx]
            if line.startswith("Monkey"):
                monkeys.append(Monkey())
                line_idx += 1
            elif "Starting items" in line:
                monkeys[-1].items = list(map(
                    int, re.findall(r"\d+", line)
                ))
                line_idx += 1
            elif "Operation" in line:
                monkeys[-1].op = line[13:]
                line_idx += 1
            elif "Test" in line:
                monkeys[-1].test = [
                    line[8:],
                    int(lines[line_idx+1][29:]),
                    int(lines[line_idx+2][30:])
                ]
                line_idx += 3
            else:
                line_idx += 1

    LOGGER.debug(
        "monkeys (after round 0):\n"
        "  %s\n",
        "\n  ".join(map(str, monkeys))
    )

    div_prod = 1
    for monkey in monkeys:
        div_prod *= int(monkey.test[0][13:])

    for _ in range(ROUNDS):
        for monkey in monkeys:
            while len(monkey.items) > 0:
                item = monkey.pop_item()
                # keep your worry levels manageable
                item = item % div_prod
                worry_level = monkey.eval_op(item)
                monkeys[monkey.get_next_monkey(worry_level)].items.append(
                    worry_level
                )

    LOGGER.debug(
        "monkeys (after round %s):\n"
        "  %s\n",
        ROUNDS, "\n  ".join(map(str, monkeys))
    )

    monkeys = list(sorted(
        monkeys,
        key=lambda m: m.inspected_items
    ))
    monkey_business = monkeys[-1].inspected_items * monkeys[-2].inspected_items
    print(f"solution: {monkey_business}")

In [6]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 2713310158
